In [7]:
import os
import pandas as pd
import numpy as np

# CSV 폴더 경로
csv_dir = "/home/avees/tee/output"

# 라벨별로 값 누적
label_data = {}

# 폴더 내 CSV 파일들 읽기
csv_files = [f for f in os.listdir(csv_dir) if f.endswith(".csv")]

for filename in sorted(csv_files):
    file_path = os.path.join(csv_dir, filename)
    try:
        df = pd.read_csv(file_path)
        for col in df.columns:
            values = df[col].dropna().astype(float).tolist()
            if not values:
                continue
            if col not in label_data:
                label_data[col] = []
            label_data[col].extend(values)
    except Exception as e:
        print(f"⚠️ {filename} 읽기 실패: {e}")

# 라벨별 통계 계산
summary_rows = []
for label, values in label_data.items():
    sorted_vals = sorted(values, reverse=True)
    max_val = sorted_vals[0]
    max2_val = sorted_vals[1] if len(sorted_vals) > 1 else np.nan
    min_val = sorted_vals[-1]
    mean_val = np.nan
    if len(sorted_vals) > 1:
        mean_val = np.mean(sorted_vals[5:])  # 최대값 제외 평균

    summary_rows.append({
        "label": label,
        "count": len(values),
        "max": max_val,
        "max2": max2_val,
        "min": min_val,
        "mean(exclude_max)": mean_val
    })

# DataFrame으로 변환 및 저장
summary_df = pd.DataFrame(summary_rows)
summary_df = summary_df.sort_values(by="label")

output_path = os.path.join(csv_dir, "output_merged.csv")
summary_df.to_csv(output_path, index=False)

print(f"\n✅ 모든 파일 통합 라벨별 통계 완료 → {output_path}")
print(f"총 {len(summary_df)}개 라벨 계산됨")



✅ 모든 파일 통합 라벨별 통계 완료 → /home/avees/tee/output/output_merged.csv
총 166개 라벨 계산됨


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# CSV 파일 경로
csv_path = "/home/avees/tee/graph/component/src/INFER.csv"

# CSV 읽기
df = pd.read_csv(csv_path, index_col=0)

# 사용할 행 선택 (순서: min → avg → max2)
selected_rows = ['min', 'avg', 'max2']
df_selected = df.loc[selected_rows]

# 라벨 (x축)
labels = df.columns.tolist()
x = np.arange(len(labels))  # x 위치

# 막대 폭
width = 0.25

plt.figure(figsize=(14, 6))

# 동일한 hue에 채도만 다른 색상
colors = ['#8ab4f8', '#1a73e8', '#003366']  # 밝은 파랑 → 중간 → 진한 네이비

# 막대 그래프
bars_min = plt.bar(x - width, df_selected.loc['min'], width,
                   label='min', color=colors[0], edgecolor='black')
bars_avg = plt.bar(x, df_selected.loc['avg'], width,
                   label='avg', color=colors[1], edgecolor='black')
bars_max2 = plt.bar(x + width, df_selected.loc['max2'], width,
                    label='max2', color=colors[2], edgecolor='black')

# avg 막대 위에 값 표시
for i, v in enumerate(df_selected.loc['avg']):
    plt.text(x[i], v + (v * 0.05), f"{v:.3f}", ha='center', va='bottom', fontsize=9)

# 시각적 설정
plt.title("INFER Component Comparison (min, avg, max2)", fontsize=14)
plt.ylabel("Execution Time (ms)", fontsize=12)
plt.xlabel("INFER Layer", fontsize=12)
plt.xticks(x, labels, rotation=45)
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# CSV 폴더 경로
csv_dir = "/home/avees/tee/graph/component/src"

# 동일한 hue에 채도만 다른 색상
colors = ['#8ab4f8', '#1a73e8', '#003366']  # 밝은 파랑 → 중간 → 진한 네이비

# 폴더 내 모든 CSV 파일 순회
for filename in sorted(os.listdir(csv_dir)):
    if not filename.endswith(".csv"):
        continue

    csv_path = os.path.join(csv_dir, filename)
    print(f"📊 Processing: {filename}")

    # CSV 읽기
    df = pd.read_csv(csv_path, index_col=0)

    # 사용할 행 선택 (순서: min → avg → max2)
    selected_rows = ['min', 'avg', 'max2']

    # 필요한 행이 모두 존재하지 않으면 건너뜀
    if not all(row in df.index for row in selected_rows):
        print(f"⚠️  Skipped {filename}: required rows missing ({selected_rows})")
        continue

    df_selected = df.loc[selected_rows]

    # 라벨 (x축)
    labels = df.columns.tolist()
    x = np.arange(len(labels))  # x 위치

    # 막대 폭
    width = 0.25

    plt.figure(figsize=(14, 6))

    # 막대 그래프
    plt.bar(x - width, df_selected.loc['min'], width,
            label='min', color=colors[0], edgecolor='black')
    plt.bar(x, df_selected.loc['avg'], width,
            label='avg', color=colors[1], edgecolor='black')
    plt.bar(x + width, df_selected.loc['max2'], width,
            label='max2', color=colors[2], edgecolor='black')

    # avg 막대 위에 값 표시
    for i, v in enumerate(df_selected.loc['avg']):
        plt.text(x[i], v + (v * 0.05), f"{v:.3f}", ha='center', va='bottom', fontsize=9)

    # 시각적 설정
    plt.title(f"{filename} (min, avg, max2)", fontsize=14)
    plt.ylabel("Execution Time (ms)", fontsize=12)
    plt.xlabel("INFER Layer", fontsize=12)
    plt.xticks(x, labels, rotation=45)
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.6)
    plt.tight_layout()

    # PNG 파일로 저장 (같은 폴더에)
    png_path = os.path.join(csv_dir, os.path.splitext(filename)[0] + ".png")
    plt.show()
    # plt.savefig(png_path, dpi=300)
    plt.close()

    print(f"✅ Saved: {png_path}")


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# CSV 폴더 경로
csv_dir = "/home/avees/tee/graph/component/src"

# 색상 설정 (dark blue 계열)
colors = ['#1a73e8', '#003366']  # 두 개 그래프용 색상

# 🔹 범례(legend) 이름 직접 지정
legend_labels = {
    "INFER.csv": "TEE",
    "LAYER.csv": "REE",
    "IN.csv": "Input",
    "OUT.csv": "Output",
}

# 파일 그룹 정의
group_pairs = [
    ("INFER.csv", "LAYER.csv", "INFER vs LAYER"),
    ("IN.csv", "OUT.csv", "IN vs OUT"),
]

for file1, file2, title in group_pairs:
    path1 = os.path.join(csv_dir, file1)
    path2 = os.path.join(csv_dir, file2)

    # 두 파일 모두 존재하는지 확인
    if not (os.path.exists(path1) and os.path.exists(path2)):
        print(f"⚠️  Skipping {title}: one or both files missing")
        continue

    # CSV 읽기
    df1 = pd.read_csv(path1, index_col=0)
    df2 = pd.read_csv(path2, index_col=0)

    # avg 행만 추출
    if 'avg' not in df1.index or 'avg' not in df2.index:
        print(f"⚠️  Skipping {title}: 'avg' row missing")
        continue

    avg1 = df1.loc['avg']
    avg2 = df2.loc['avg']

    # x축: Layer index (숫자)
    x1 = np.arange(len(avg1))
    x2 = np.arange(len(avg2))

    plt.figure(figsize=(14, 6))

    # 🔹 legend 이름 지정
    label1 = legend_labels.get(file1, file1.replace('.csv', ''))
    label2 = legend_labels.get(file2, file2.replace('.csv', ''))

    # 꺾은선그래프
    plt.plot(x1, avg1.values, marker='o', linestyle='-', color=colors[0], linewidth=2, label=label1)
    plt.plot(x2, avg2.values, marker='s', linestyle='-', color=colors[1], linewidth=2, label=label2)

    # 각 점 위에 값 표시
    for i, v in enumerate(avg1.values):
        plt.text(i, v + (v * 0.02), f"{v:.3f}", ha='center', fontsize=8, color=colors[0])
    for i, v in enumerate(avg2.values):
        plt.text(i, v + (v * 0.02), f"{v:.3f}", ha='center', fontsize=8, color=colors[1])

    # 시각적 설정
    plt.title(f"{title}", fontsize=14)
    plt.xlabel("Layer Index", fontsize=12)
    plt.ylabel("Execution Time (ms)", fontsize=12)
    plt.xticks(x1)  # x축을 0,1,2,... 형태로 표시
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
import os
import pandas as pd

# CSV 폴더 경로
csv_dir = "/home/avees/tee/output/tee_time"
output_path = os.path.join(csv_dir, "merged.csv")

# 모든 CSV를 담을 리스트
all_dfs = []

# 디렉토리 내 모든 CSV 파일 순회
for filename in sorted(os.listdir(csv_dir)):
    if filename.endswith(".csv"):
        csv_path = os.path.join(csv_dir, filename)
        df = pd.read_csv(csv_path)

        # 파일명(확장자 제외) 저장용 열 추가
        df.insert(0, "source", os.path.splitext(filename)[0])

        # DataFrame 리스트에 추가
        all_dfs.append(df)

# 모든 CSV 병합 (아래로 concat)
merged_df = pd.concat(all_dfs, ignore_index=True)

# 병합된 CSV 저장
merged_df.to_csv(output_path, index=False)

print(f"✅ 모든 CSV 파일을 병합하여 저장했습니다: {output_path}")
print(f"총 행 개수: {len(merged_df)}")


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# CSV 폴더 경로
csv_dir = "/home/avees/tee/graph/component/src"

# 색상 설정 (dark blue 계열)
colors = ['#1a73e8', '#003366']  # INFER/LAYER, IN/OUT 각각 구분용

# 파일 그룹 정의
group_pairs = [
    ("INFER.csv", "LAYER.csv", "INFER vs LAYER"),
]

for file1, file2, title in group_pairs:
    path1 = os.path.join(csv_dir, file1)
    path2 = os.path.join(csv_dir, file2)

    # 두 파일 모두 존재하는지 확인
    if not (os.path.exists(path1) and os.path.exists(path2)):
        print(f"⚠️  Skipping {title}: one or both files missing")
        continue

    # CSV 읽기
    df1 = pd.read_csv(path1, index_col=0)
    df2 = pd.read_csv(path2, index_col=0)

    # avg 행만 추출
    if 'avg' not in df1.index or 'avg' not in df2.index:
        print(f"⚠️  Skipping {title}: 'avg' row missing")
        continue

    avg1 = df1.loc['avg']
    avg2 = df2.loc['avg']

    # 그래프 설정
    plt.figure(figsize=(12, 6))
    x1 = np.arange(len(avg1))
    x2 = np.arange(len(avg2))

    plt.plot(x1, avg1.values-0.065, marker='o', linestyle='-', color=colors[0], label=file1.replace('.csv', ''))
    plt.plot(x2, avg2.values, marker='s', linestyle='-', color=colors[1], label=file2.replace('.csv', ''))
    plt.plot(x1, (avg1.values-0.068) / avg2.values, marker='o', linestyle='-', color=colors[0], label=file1.replace('.csv', ''))

    # 값 표시
    for i, v in enumerate(avg1.values):
        plt.text(i, v + (v * 0.02), f"{v-0.068:.3f}", ha='center', fontsize=8, color=colors[0])
    for i, v in enumerate(avg2.values):
        plt.text(i, v + (v * 0.02), f"{v:.3f}", ha='center', fontsize=8, color=colors[1])

    # 시각적 설정
    plt.title(f"{title} (avg only)", fontsize=14)
    plt.xlabel("Layer Index", fontsize=12)
    plt.ylabel("Execution Time (ms)", fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.legend()
    plt.tight_layout()
    plt.show()
